In [85]:
import numpy as np
import math


In [86]:
def _closest_pow2(n):
    return 2**(math.ceil(math.log(n, 2)))

def _fft_rec(t,K):
    ret = []
    
    if K == 0:
        return t
    else:
        evens, odds = t[::2],t[1::2]
        rec_evens = _fft_rec(evens,K-1)
        rec_odds = _fft_rec(odds,K-1)
        a_len = int(2**(K-1))
        
        for i in range(len(t)):
            n =  rec_evens[i % a_len] + np.exp((2j*math.pi * i/2**K)) * rec_odds[i % a_len]
            ret.append(np.around(n, 3))
        
        return ret
    
def fft(t):
    t_len = len(t)
    
    next_pow = _closest_pow2(t_len)
    k = math.log(next_pow,2)
    num_add_zeros = next_pow - t_len
    zeros_array = np.array([0]*num_add_zeros)
    complete_array = np.concatenate((t,zeros_array),axis=0)

    return np.array(_fft_rec(complete_array,k))


def invert_fft(t, fft_func=fft):
    t_len = len(t)
    
    next_pow = _closest_pow2(t_len)
    k = math.log(next_pow,2)
    num_add_zeros = next_pow - t_len
    zeros_array = np.array([0]*num_add_zeros)
    complete_array = np.concatenate((t,zeros_array),axis=0)
    
    t_conj = [np.conj(n) for n in complete_array]
    
    dft = fft_func(t_conj)
    
    dft_conj = [np.conj(n) for n in dft]
    
    return [n/len(t) for n in dft_conj]

In [87]:
a = np.array([1,2,3,4])

#s = fft(a)
t = invert_fft(a)

print(t)


[(2.5-0j), (-0.5+0.5j), (-0.5+0j), (-0.5-0.5j)]


In [88]:
b = list(np.random.randint(low=0, high=5, size=15))
b.append(10)
b

[3, 0, 0, 4, 3, 3, 1, 1, 3, 3, 2, 2, 4, 1, 4, 10]

In [263]:
def rand_polinomio(long=2**10,base=10):
    ret = list(np.random.randint(low=0, high=base, size=long-1).astype('uint8'))
    ret.append(np.random.randint(low=1, high=base))
    return ret

def poli_2_num(l_pol,base=10):
    ret = 0
    
    for i in range(len(l_pol)):
        ret += int(l_pol[i]*pow(base, i))
    return ret


l = rand_polinomio()
#print(l)
print(poli_2_num(l, 2))

1263957871363095120965027680716536665619941980592286287645898217987630928734350044113700118547167345210397419465864315335982064674075563229273221534500490949176336413318486994213861775859230248552781554268682860210778724064251481901116133518998680286191613018003077206699022853277494208576370386923592853626552


In [286]:
def rand_numero(num_digits,base=10):
    poli = rand_polinomio(num_digits, base)
    print(poli)
    
    return poli_2_num(poli, base)

def num_2_poli(num,base=10):
        
    poli = []
    while True:
        remain = num % base
        poli.append(remain)
        num = math.floor(num / base)
        if num == 0:
            break

    return poli


print(rand_numero(5))
print(num_2_poli(1801, 5))

[9, 4, 7, 7, 7]
77749
[1, 0, 2, 4, 2]


In [287]:
def mult_polinomios(l_pol_1, l_pol_2):
    prod = [0]*(len(l_pol_1)+len(l_pol_2)-1)
    
    for i in range(len(l_pol_1)):
        for j in range(len(l_pol_2)):
            prod[i+j] = prod[i+j] + l_pol_1[i] * l_pol_2[j]
    
    return prod

def mult_polinomios_fft(l_pol_1, l_pol_2, fft_func=fft):
    pass

l_a = [1, 3]
l_b = [1, 0, 0, 4]
print(mult_polinomios(l_a, l_b))

[1, 3, 0, 4, 12]
